# Описание

Один из возможных источников данных с адресами -- ГАР от ФИАС (https://fias.nalog.ru/Frontend). Однако здесь данные и избыточные для наших целей, и лежат в специфическом формате и скачиваются все одним архивом (размеров в сжатом виде в десятки, а в распакованном -- в сотни гигабайт). Поэтому посмотрим сперва на эти данные и поймём, как их можно обработать для наших целей.

Предполагается, что мы заранее скачали архив по ссылке выше.

# Импорты

In [1]:
import zipfile
import os
from io import StringIO

import pandas as pd

In [2]:
path_to_archive = "E:\\!Storage\\ГАР\\gar_xml.zip"

# Данные

In [4]:
z = zipfile.ZipFile(path_to_archive) 

In [5]:
z_files = sorted(z.namelist())

In [6]:
def load_dataset(zip_file, zip_path, key_col=None, end_col="ENDDATE"):
    actual_col = "ISACTUAL"
    ds = pd.read_xml(zip_file.open(zip_path), parser='etree')
    ds_filtered = ds
    if actual_col in ds:
        ds_filtered = ds[ds[actual_col] == 1]
    if (key_col is None) or (end_col not in ds):
        return ds_filtered
    return (
        ds_filtered
        .sort_values(end_col, ascending=False)
        .groupby(key_col, as_index=False)
        .first()
    )

In [7]:
root_files = [f for f in z_files if os.path.dirname(f) == ""]

## Описание полей

In [8]:
root_files

['AS_ADDHOUSE_TYPES_20250327_adbcfb0e-1d7f-4988-9355-44cbe3dd151f.XML',
 'AS_ADDR_OBJ_TYPES_20250327_489d0f3f-f003-4017-9031-82f8f4b96672.XML',
 'AS_APARTMENT_TYPES_20250327_ce5f2321-f6ee-4f47-b92e-fff9dd89f324.XML',
 'AS_HOUSE_TYPES_20250327_bdc6731e-d51b-492c-aab4-3df2b1dca179.XML',
 'AS_NORMATIVE_DOCS_KINDS_20250327_72ab1bd8-7ea9-4519-b16e-b74ec6d8fc86.XML',
 'AS_NORMATIVE_DOCS_TYPES_20250327_b6e368b1-6bfb-422f-a92a-3824bb3d3d72.XML',
 'AS_OBJECT_LEVELS_20250327_654536f3-fb92-4d71-ad62-86d4c405638c.XML',
 'AS_OPERATION_TYPES_20250327_f2eafdcb-a047-472d-bb52-e130a3ad56f8.XML',
 'AS_PARAM_TYPES_20250327_e0e1ab39-c33f-469c-8aaf-d7fd2e087578.XML',
 'AS_ROOM_TYPES_20250327_5cc957c5-4332-4072-b1b0-4fbfab53f3b9.XML',
 'version.txt']

In [9]:
path = root_files[0]
print(path)
load_dataset(z, path)

AS_ADDHOUSE_TYPES_20250327_adbcfb0e-1d7f-4988-9355-44cbe3dd151f.XML


,ID,NAME,SHORTNAME,DESC,ISACTIVE,UPDATEDATE,STARTDATE,ENDDATE
0,1,Корпус,к.,Корпус,True,2015-12-25,2015-12-25,2079-06-06
1,2,Строение,стр.,Строение,True,2015-12-25,2015-12-25,2079-06-06
2,3,Сооружение,соор.,Сооружение,True,2015-12-25,2015-12-25,2079-06-06
3,4,Литера,литера,Литера,False,1900-01-01,1900-01-01,2015-12-25


In [10]:
path = root_files[1]
print(path)
load_dataset(z, path)

AS_ADDR_OBJ_TYPES_20250327_489d0f3f-f003-4017-9031-82f8f4b96672.XML


,ID,LEVEL,NAME,SHORTNAME,DESC,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE
0,5,1,Автономная область,Аобл,Автономная область,1900-01-01,1900-01-01,2015-11-05,True
1,2,1,Автономная область,а.обл.,Автономная область,2015-11-05,2015-11-05,2079-06-06,True
2,4,1,Автономный округ,АО,Автономный округ,1900-01-01,1900-01-01,2015-11-05,True
3,3,1,Автономный округ,а.окр.,Автономный округ,2015-11-05,2015-11-05,2079-06-06,True
4,6,1,Город,г,Город,1900-01-01,1900-01-01,2015-11-05,True
...,...,...,...,...,...,...,...,...,...
418,410,16,Участок,уч-к,Участок,1900-01-01,1900-01-01,2015-12-31,True
419,411,16,Ферма,ферма,Ферма,1900-01-01,1900-01-01,2015-12-31,True
420,412,16,Хутор,х,Хутор,1900-01-01,1900-01-01,2079-06-06,True
421,413,16,Шоссе,ш,Шоссе,1900-01-01,1900-01-01,2079-06-06,True


In [11]:
path = root_files[3]
print(path)
load_dataset(z, path)

AS_HOUSE_TYPES_20250327_bdc6731e-d51b-492c-aab4-3df2b1dca179.XML


,ID,NAME,SHORTNAME,DESC,ISACTIVE,UPDATEDATE,STARTDATE,ENDDATE
0,1,Владение,влд.,Владение,False,1900-01-01,1900-01-01,2015-11-05
1,2,Дом,д.,Дом,True,1900-01-01,1900-01-01,2079-06-06
2,3,Домовладение,двлд.,Домовладение,False,1900-01-01,1900-01-01,2015-11-05
3,4,Гараж,г-ж,Гараж,True,1900-01-01,1900-01-01,2079-06-06
4,5,Здание,зд.,Здание,True,1900-01-01,1900-01-01,2079-06-06
5,6,Шахта,шахта,Шахта,True,1900-01-01,1900-01-01,2079-06-06
6,7,Строение,стр.,Строение,True,1900-01-01,1900-01-01,2079-06-06
7,8,Сооружение,соор.,Сооружение,True,1900-01-01,1900-01-01,2079-06-06
8,9,Литера,литера,Литера,False,1900-01-01,1900-01-01,2015-12-25
9,10,Корпус,к.,Корпус,True,1900-01-01,1900-01-01,2079-06-06


In [12]:
path = root_files[6]
print(path)
load_dataset(z, path)

AS_OBJECT_LEVELS_20250327_654536f3-fb92-4d71-ad62-86d4c405638c.XML


,LEVEL,NAME,STARTDATE,ENDDATE,UPDATEDATE,ISACTIVE
0,1,Субъект РФ,1900-01-01,2079-06-06,1900-01-01,True
1,2,Административный район,1900-01-01,2079-06-06,1900-01-01,True
2,3,Муниципальный район,1900-01-01,2079-06-06,1900-01-01,True
3,4,Сельское/городское поселение,1900-01-01,2079-06-06,1900-01-01,True
4,5,Город,1900-01-01,2079-06-06,1900-01-01,True
5,6,Населенный пункт,1900-01-01,2079-06-06,1900-01-01,True
6,7,Элемент планировочной структуры,1900-01-01,2079-06-06,1900-01-01,True
7,8,Элемент улично-дорожной сети,1900-01-01,2079-06-06,1900-01-01,True
8,9,Земельный участок,1900-01-01,2079-06-06,1900-01-01,True
9,10,"Здание (строение), сооружение",1900-01-01,2079-06-06,1900-01-01,True


In [13]:
path = root_files[7]
print(path)
load_dataset(z, path)

AS_OPERATION_TYPES_20250327_f2eafdcb-a047-472d-bb52-e130a3ad56f8.XML


,ID,NAME,ISACTIVE,UPDATEDATE,STARTDATE,ENDDATE,SHORTNAME
0,0,Не определено,True,1900-01-01,1900-01-01,2079-06-06,None
1,1,Инициация,True,1900-01-01,1900-01-01,2079-06-06,None
2,10,Добавление,True,1900-01-01,1900-01-01,2079-06-06,None
3,20,Редактирование,True,1900-01-01,1900-01-01,2079-06-06,None
4,21,Групповое изменение,True,1900-01-01,1900-01-01,2079-06-06,Групповое изменение
5,23,Редактирование протяжённых объектов,True,1900-01-01,1900-01-01,2079-06-06,Редактирование протяжённых объектов
6,24,Групповое редактирование объектов,True,1900-01-01,1900-01-01,2079-06-06,Групповое редактирование объектов
7,25,Размещение сведений о почтовых индексах на осн...,True,1900-01-01,1900-01-01,2079-06-06,Размещение сведений о почтовых индексах на осн...
8,26,Обновление справочника ОКТМО,True,1900-01-01,1900-01-01,2079-06-06,Обновление справочника ОКТМО
9,27,Обновление справочника ОКАТО,True,1900-01-01,1900-01-01,2079-06-06,Обновление справочника ОКАТО


In [14]:
path = root_files[8]
print(path)
load_dataset(z, path)

AS_PARAM_TYPES_20250327_e0e1ab39-c33f-469c-8aaf-d7fd2e087578.XML


,ID,NAME,DESC,CODE,ISACTIVE,UPDATEDATE,STARTDATE,ENDDATE
0,1,ИФНС ФЛ,ИФНС ФЛ,IFNSFL,True,2018-06-15,2011-11-01,2079-06-06
1,2,ИФНС ЮЛ,ИФНС ЮЛ,IFNSUL,True,2018-06-15,2011-11-01,2079-06-06
2,3,ИФНС ФЛ ТЕР УЧ,Территориальный участок ИФНС ФЛ,territorialifnsflcode,True,2018-06-15,2011-11-01,2079-06-06
3,4,ИФНС ЮЛ ТЕР УЧ,Территориальный участок ИФНС ЮЛ,territorialifnsulcode,True,2018-06-15,2011-11-01,2079-06-06
4,5,Почтовый индекс,Информация о почтовом индексе,PostIndex,True,2018-06-15,2011-11-01,2079-06-06
5,6,ОКАТО,ОКАТО,OKATO,True,2018-06-19,2011-11-01,2079-06-06
6,7,OKTMO,OKTMO,OKTMO,True,2018-06-19,2011-11-01,2079-06-06
7,8,Кадастровый номер,Кадастровый номер,CadastrNum,True,2018-06-19,2011-11-01,2079-06-06
8,10,Код КЛАДР,Код адресного объекта одной строкой с признако...,CODE,True,2018-06-21,2011-11-01,2079-06-06
9,11,Код КЛАДР без признака актуальности,Код адресного объекта одной строкой без призна...,PLAINCODE,True,2018-06-21,2011-11-01,2079-06-06


In [15]:
path = root_files[9]
print(path)
load_dataset(z, path)

AS_ROOM_TYPES_20250327_5cc957c5-4332-4072-b1b0-4fbfab53f3b9.XML


,ID,NAME,DESC,ISACTIVE,STARTDATE,ENDDATE,UPDATEDATE,SHORTNAME
0,0,Не определено,Не определено,True,1900-01-01,2015-11-05,2011-01-01,None
1,1,Комната,Комната,True,1900-01-01,2079-06-06,2011-01-01,ком.
2,2,Помещение,Помещение,True,1900-01-01,2079-06-06,2011-01-01,помещ.


## Изучение региона

Начнём с Москвы, как наиболее известного мне региона.

In [13]:
region_id = "77"

In [14]:
region_files = [f for f in z_files if os.path.dirname(f) == region_id]

In [15]:
region_files

['77/AS_ADDR_OBJ_20250327_0fa4f946-e26b-435e-a465-6356525f2841.XML',
 '77/AS_ADDR_OBJ_DIVISION_20250327_9e8334bd-b318-4ad9-b616-0d3ae0e08a56.XML',
 '77/AS_ADDR_OBJ_PARAMS_20250327_2255531a-48ee-46af-86b0-ffe50c3e8dee.XML',
 '77/AS_ADM_HIERARCHY_20250327_8ff210e4-2ec8-4f7b-801e-416214445482.XML',
 '77/AS_APARTMENTS_20250327_29eafc2e-0d45-4083-8b30-6409a9cb6d5e.XML',
 '77/AS_APARTMENTS_PARAMS_20250327_2832bf30-c3d0-48a5-9e8d-68bbaeff2099.XML',
 '77/AS_CARPLACES_20250327_b4708798-9b61-4287-9b00-00cf3ebd6f1b.XML',
 '77/AS_CARPLACES_PARAMS_20250327_8837e9cb-974d-4cca-99d8-56c0c4e23bfb.XML',
 '77/AS_CHANGE_HISTORY_20250327_ee6555d3-5520-4f12-9e8f-e87bb9bd6ec7.XML',
 '77/AS_HOUSES_20250327_5b78ff2c-2bdc-4d9c-a90e-52251767f462.XML',
 '77/AS_HOUSES_PARAMS_20250327_592a46df-5bd7-440f-9dfe-3528dcea5af3.XML',
 '77/AS_MUN_HIERARCHY_20250327_5aa56be4-6f00-48ce-b2ab-0bd07e705e3e.XML',
 '77/AS_NORMATIVE_DOCS_20250327_212c647f-bb6d-40f1-a2cc-001c657f591c.XML',
 '77/AS_REESTR_OBJECTS_20250327_bf484da9-d

### Адресные объекты

В `AS_ADDR_OBJ` лежат объекты, составляющие элементы адреса -- от региона и до улицы.

In [16]:
path = region_files[0]
print(path)
address_parts_dataset = load_dataset(z, path)

77/AS_ADDR_OBJ_20250327_0fa4f946-e26b-435e-a465-6356525f2841.XML


In [17]:
address_parts_dataset.shape

(13313, 15)

In [18]:
address_parts_dataset.head()

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
1,52102410,1406821,c4c90184-8410-465b-938d-cdd362d241c3,591625953,Конструктора Лукина,ул,8,40,1731422,NaN,2024-03-13,2024-03-13,2079-06-06,1,1
2,52150772,1412644,0de928f6-3a67-497f-bb3d-d98d4785a2be,638391439,108,кв-л,7,42,1738646,52150771.0,2025-02-05,2025-02-05,2079-06-06,1,0
5,52150816,170040574,c6fbfb5b-6739-4c31-b84b-e5caaf73e85f,638395088,131А,кв-л,7,40,52150624,NaN,2025-02-05,2025-02-05,2079-06-06,1,1
7,52150843,170042113,392525b8-aa87-46fb-ae8e-382116530438,638398563,288,кв-л,7,40,52150671,NaN,2025-02-05,2025-02-05,2079-06-06,1,1
8,52150858,1414020,3dc69326-4787-4992-b87c-0e34e742bf09,638399264,30,кв-л,7,42,1740499,52150857.0,2025-02-05,2025-02-05,2079-06-06,1,0


In [138]:
address_parts_dataset.groupby(["ISACTUAL", "ISACTIVE"]).count()

ID  OBJECTID  OBJECTGUID  CHANGEID  NAME  TYPENAME  \
ISACTUAL ISACTIVE                                                         
1        0         4484      4484        4484      4484  4484      4484   
         1         8829      8829        8829      8829  8829      8829   

                   LEVEL  OPERTYPEID  PREVID  NEXTID  UPDATEDATE  STARTDATE  \
ISACTUAL ISACTIVE                                                             
1        0          4484        4484    4484    4482        4484       4484   
         1          8829        8829    8829    6666        8829       8829   

                   ENDDATE  
ISACTUAL ISACTIVE           
1        0            4484  
         1            8829

In [140]:
address_parts_dataset[address_parts_dataset["LEVEL"] == 1]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
9068,1729288,1405113,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,3846120,Москва,г,1,1,0,0.0,2018-09-01,1900-01-01,2079-06-06,1,1


In [141]:
for level in sorted(address_parts_dataset.LEVEL.unique()):
    types = sorted(address_parts_dataset[address_parts_dataset["LEVEL"] == level]["TYPENAME"].unique())
    print(level)
    print(types)

1
['г']
2
['п', 'р-н']
3
['вн.тер.г.']
5
['г', 'г.', 'тер']
6
['д', 'дп', 'кв-л', 'нп', 'п', 'п/о', 'промзона', 'рп', 'с', 'тер', 'х']
7
['гск', 'днп', 'кв-л', 'м-ко', 'мкр', 'мкр.', 'н/п', 'наб.', 'парк', 'сквер', 'снт', 'тер', 'тер.', 'тер. ДНП', 'тер. ДНТ', 'тер. ДПК', 'тер. СНТ', 'тер. СПК', 'тер. ТСН', 'ш.']
8
['ал.', 'аллея', 'б-р', 'гск', 'д', 'дор', 'кв-л', 'км', 'линия', 'лн.', 'мкр', 'наб', 'наб.', 'нп', 'п', 'парк', 'пер', 'пер.', 'пл', 'пл.', 'платф', 'пр-д', 'пр-кт', 'проезд', 'просек', 'просека', 'проселок', 'проулок', 'с-р', 'снт', 'ст', 'стр', 'тер', 'туп', 'туп.', 'ул', 'ул.', 'уч-к', 'х', 'ш', 'ш.']
15
['гск', 'промзона', 'снт']


In [240]:
address_parts_dataset.groupby(
    "LEVEL"
)["ID"].count()

LEVEL
1        1
2       28
3      151
5        5
6      409
7     5882
8     6829
15       8
Name: ID, dtype: int64

In [142]:
address_parts_dataset[address_parts_dataset["NAME"].str.contains("Ленин")]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
481,1727092,1403313,9c3e9392-0324-4d21-9cf5-70076f1b5e15,3841514,Ленинградский,пр-кт,8,1,0,0.0,2016-05-17,1900-01-01,2079-06-06,1,1
553,1075954,870463,f168f4da-a379-46df-b4ce-0cf421ad7475,2407755,Ленина,ул,8,1,0,0.0,2017-09-07,1900-01-01,2079-06-06,1,1
1253,1740905,1414349,c5fec714-04f5-4cb3-89d8-d7cec97116e6,3865610,Ленинградское,ш,8,1,0,0.0,2016-05-11,1900-01-01,2079-06-06,1,1
5924,1728930,1404818,91ab1cb1-d0a4-489c-9182-9dfbabcd9e2b,3845389,Лениногорская,ул,8,1,0,0.0,2016-05-31,1900-01-01,2079-06-06,1,1
7080,1863791,867679,7a8015ad-a3eb-4fd0-b4c4-c34ef55bb294,156242574,Ленина,ул,8,30,1071404,0.0,2020-09-03,2020-09-03,2079-06-06,1,0
7605,52126950,1412966,e703d5fd-4655-48c9-8768-51a3818ebab1,601222326,СНТ Ленинское Знамя,тер,7,30,1739141,0.0,2024-09-13,2024-09-13,2079-06-06,1,0
8925,1732570,1407748,4350977e-e152-4018-a33d-b13143bbecda,3852507,Ленина,ул,8,1,0,0.0,2016-10-05,1900-01-01,2079-06-06,1,1
9238,1726229,1402585,5f2a1243-a57b-418e-baee-ff76f4993b45,3839686,Ленинский,пр-кт,8,1,0,0.0,2016-05-17,1900-01-01,2079-06-06,1,1
9952,1963016,1405058,12bf2472-62ee-4db6-b998-565141e7a9d1,289726664,Ленинские Горы,ул,8,30,1729220,0.0,2022-09-08,2022-09-08,2079-06-06,1,0
10778,1739844,1413526,0faa0cfa-dfe8-4f5a-a001-2727a41d7f21,3864181,Ленина,ул,8,1,0,0.0,2016-06-15,1900-01-01,2079-06-06,1,1


In [144]:
address_parts_dataset.groupby("OBJECTID").count()["ID"].value_counts()

ID
1    13313
Name: count, dtype: int64

### Дома

In [151]:
path = region_files[9]
print(path)
houses_dataset = load_dataset(z, path)

77/AS_HOUSES_20250327_5b78ff2c-2bdc-4d9c-a90e-52251767f462.XML


In [152]:
houses_dataset.shape

(347713, 18)

In [154]:
houses_dataset.sample(10)

,ID,OBJECTID,OBJECTGUID,CHANGEID,HOUSENUM,ADDNUM1,HOUSETYPE,ADDTYPE1,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE,ADDNUM2,ADDTYPE2
279655,48951830,80896577,1cee6571-6b4e-41d3-a42b-3a462b8c3027,120227288,12,1,2.0,2.0,10,0,0.0,2018-04-27,2018-04-27,2079-06-06,1,1,None,NaN
14340,37425008,62272596,b2014108-aa47-43ca-b38a-bec470e0dfd3,92979928,9,3,2.0,1.0,10,0,0.0,2017-12-18,1900-01-01,2079-06-06,1,1,None,NaN
192089,123551283,56266558,5c856f8a-0427-4ef1-9eec-78f9347091aa,456252075,12,1,2.0,2.0,30,33661148,0.0,2022-10-17,2022-10-17,2079-06-06,1,0,None,NaN
283385,19723396,33167239,56ee3d84-5064-4493-8896-5683592ca258,50581778,31,14,2.0,2.0,10,0,0.0,2011-11-24,1900-01-01,2079-06-06,1,1,None,NaN
295732,70220101,99508452,6bc996c3-1869-4b6e-b5a4-9034f3aa1b69,162786672,8А,None,2.0,NaN,10,0,0.0,2020-12-09,2020-12-09,2079-06-06,1,1,None,NaN
247898,50546568,83504973,3efa88c4-a6e1-4bf2-8f63-a9fa3c08a94a,124044336,104,1,1.0,2.0,10,0,0.0,2019-04-08,2019-03-26,2079-06-06,1,1,None,NaN
170570,123906192,78156503,d8c0e1e9-3049-40dc-9eec-f89e289d2356,488529487,8,17,2.0,2.0,30,47219618,0.0,2022-12-01,2022-12-01,2079-06-06,1,0,None,NaN
14939,8433792,15011701,8b870254-58ba-4017-9159-5257a259555a,23951343,17А,None,2.0,NaN,10,0,0.0,2012-02-27,1900-01-01,2079-06-06,1,1,None,NaN
463645,26891102,45024461,e48f38df-03c4-4865-9b4d-0002eb923f99,67845777,107,None,3.0,NaN,30,26891094,0.0,2019-06-14,2019-06-11,2019-06-11,1,0,None,NaN
52008,13536471,22947336,6dbda1af-2dbe-4a0e-8910-4fbdf530dff5,35711962,None,598,NaN,2.0,30,13536100,0.0,2019-09-11,2019-09-10,2019-09-10,1,0,None,NaN


In [155]:
houses_dataset.groupby("OBJECTID").count()["ID"].value_counts()

ID
1    347713
Name: count, dtype: int64

### Административная иерархия

In [192]:
path = region_files[3]
print(path)
adm_dataset = load_dataset(z, path, "OBJECTID")

77/AS_ADM_HIERARCHY_20250327_8ff210e4-2ec8-4f7b-801e-416214445482.XML


In [157]:
adm_dataset.shape

(8514060, 17)

In [158]:
adm_dataset.sample(10)

,ID,OBJECTID,PARENTOBJID,CHANGEID,REGIONCODE,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH,AREACODE,CITYCODE,PLACECODE,PLANCODE,STREETCODE
4904264,99311830,28698223,28695987,44105360,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1414511.28695987.28698223,NaN,NaN,NaN,NaN,NaN
2674048,95758180,18439422,18427920,29021221,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1405805.18427920.18439422,NaN,NaN,NaN,NaN,NaN
312245,78751483,17257006,17236946,27276749,77,0,0,2012-07-01,2012-07-01,2012-07-01,0,1405113.811011.812759.17236946.17257006,NaN,NaN,NaN,NaN,NaN
5719414,116594165,99399751,71536994,161335724,77,0,0,2020-12-02,2020-12-02,2079-06-06,1,1405113.1409379.71536994.99399751,NaN,NaN,NaN,NaN,NaN
7486766,239988941,164109033,164103251,614020083,77,234338442,240657812,2025-01-17,2024-12-27,2025-01-17,0,1405113.105530134.164103251.164109033,0.0,0.0,0.0,0.0,1332.0
2004903,94369659,59212437,59210833,88455689,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1405171.59210833.59212437,NaN,NaN,NaN,NaN,NaN
5592629,114732482,97627483,84228669,153832216,77,0,0,2020-07-23,2020-07-23,2079-06-06,1,1405113.1402585.84228669.97627483,NaN,NaN,NaN,NaN,NaN
3914921,97275686,67879448,67866019,101192993,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1411492.67866019.67879448,NaN,NaN,NaN,NaN,NaN
777376,90977175,62467243,62461753,93256664,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1400892.62461753.62467243,NaN,NaN,NaN,NaN,NaN
980826,91522633,51480313,51442391,77177736,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1401048.51442391.51480313,NaN,NaN,NaN,NaN,NaN


Рассмотрим улицу Дмитрия Ульянова.

In [195]:
object_id = 1406744

In [196]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == object_id]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
18855,1731326,1406744,5c98ab12-a4e9-4a94-a08c-5c41469aeb1f,3850156,Дмитрия Ульянова,ул,8,1,0,0.0,2016-06-10,1900-01-01,2079-06-06,1,1


Её родитель в иерархии -- город Москва.

In [197]:
adm_dataset[adm_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,REGIONCODE,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH,AREACODE,CITYCODE,PLACECODE,PLANCODE,STREETCODE
4742,1406744,114980138,1405113,154408041,77,95328001,0,2020-08-11,2020-08-11,2079-06-06,1,1405113.1406744,0.0,0.0,0.0,0.0,2964.0


In [194]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == 1405113]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
9068,1729288,1405113,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,3846120,Москва,г,1,1,0,0.0,2018-09-01,1900-01-01,2079-06-06,1,1


Детей у неё много:

In [198]:
adm_dataset[adm_dataset["PARENTOBJID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,REGIONCODE,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH,AREACODE,CITYCODE,PLACECODE,PLANCODE,STREETCODE
1620692,31075780,95328007,1406744,47547182,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1406744.31075780,NaN,NaN,NaN,NaN,NaN
1620862,31081014,95328055,1406744,47554656,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1406744.31081014,NaN,NaN,NaN,NaN,NaN
1622223,31108141,95328300,1406744,47592246,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1406744.31108141,NaN,NaN,NaN,NaN,NaN
1622660,31114645,95328364,1406744,47602143,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1406744.31114645,NaN,NaN,NaN,NaN,NaN
1622685,31115172,95328366,1406744,47603046,77,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.1406744.31115172,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6539197,164953656,235321426,1406744,600196914,77,0,0,2024-08-30,2024-08-30,2079-06-06,1,1405113.1406744.164953656,0.0,0.0,0.0,0.0,2964.0
6539208,164953939,235321709,1406744,600197809,77,0,0,2024-08-30,2024-08-30,2079-06-06,1,1405113.1406744.164953939,0.0,0.0,0.0,0.0,2964.0
6539209,164953950,235321720,1406744,600197824,77,0,0,2024-08-30,2024-08-30,2079-06-06,1,1405113.1406744.164953950,0.0,0.0,0.0,0.0,2964.0
6539210,164953951,235321721,1406744,600197825,77,0,0,2024-08-30,2024-08-30,2079-06-06,1,1405113.1406744.164953951,0.0,0.0,0.0,0.0,2964.0


Часть из них -- дома.

In [200]:
houses_dataset[houses_dataset["OBJECTID"] == 31075780]

,ID,OBJECTID,OBJECTGUID,CHANGEID,HOUSENUM,ADDNUM1,HOUSETYPE,ADDTYPE1,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE,ADDNUM2,ADDTYPE2
407579,18446079,31075780,984e82d0-f541-4c97-8812-63d0972e8aba,47547182,12,1,2.0,1.0,10,0,0.0,2019-02-23,1900-01-01,2079-06-06,1,1,None,NaN


А вот часть -- что-то непонятное.

In [199]:
houses_dataset[houses_dataset["OBJECTID"] == 164953952]

,ID,OBJECTID,OBJECTGUID,CHANGEID,HOUSENUM,ADDNUM1,HOUSETYPE,ADDTYPE1,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE,ADDNUM2,ADDTYPE2


А все ли дома есть в этой иерарахии? Да.

In [207]:
houses_dataset["OBJECTID"].isin(adm_dataset["OBJECTID"]).sum() / len(houses_dataset)

np.float64(1.0)

### Муниципальная иерархия

In [209]:
%%time
path = region_files[11]
print(path)
mun_dataset = load_dataset(z, path, "OBJECTID")

77/AS_MUN_HIERARCHY_20250327_5aa56be4-6f00-48ce-b2ab-0bd07e705e3e.XML
CPU times: total: 4min 5s
Wall time: 4min 7s


In [215]:
mun_dataset.shape

(6733085, 12)

In [212]:
mun_dataset.sample(10)

,OBJECTID,ID,PARENTOBJID,CHANGEID,OKTMO,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH
6379618,162029989,242893101,856205.0,619743036,45963000,230647715,0,2025-01-27,2025-01-27,2079-06-06,1,1405113.169225309.856205.162029989
778436,16065471,56927658,16064273.0,25524832,45340000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251332.1409529.16064273.16065471
4365594,76154621,58437235,76150228.0,113282418,45305000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251250.1418586.76150228.76154621
413646,9393959,54317783,9391924.0,15651623,45338000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251236.1399289.9391924.9393959
4205357,72859209,58206408,72854703.0,108492749,45328000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251283.1415104.72854703.72859209
1083671,21347462,54226810,21339695.0,33344609,45320000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251379.1400143.21339695.21347462
6245376,160252159,240031821,160243856.0,614042523,45941000,228327855,0,2024-12-27,2024-12-27,2079-06-06,1,1405113.95251378.103564290.160243856.160252159
6208660,159617149,227679674,159595539.0,507720805,45365000,0,0,2023-08-08,2023-08-08,2079-06-06,1,1405113.95251305.1410543.159595539.159617149
1631858,31365904,57753639,31364281.0,47975740,45301000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251292.1412327.31364281.31365904
3179134,58231004,54401369,58216714.0,87037646,45380000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251325.1402317.58216714.58231004


Рассмотрим опять улицу Дмитрия Ульянова.

In [213]:
object_id = 1406744

In [214]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == object_id]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
18855,1731326,1406744,5c98ab12-a4e9-4a94-a08c-5c41469aeb1f,3850156,Дмитрия Ульянова,ул,8,1,0,0.0,2016-06-10,1900-01-01,2079-06-06,1,1


Теперь у неё родитель не сразу город, а муниципальный округ (а где район?).

In [216]:
mun_dataset[mun_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,OKTMO,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH
4587,1406744,55907563,95251310.0,48103478,45397000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251310.1406744


In [217]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == 95251310]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
724,1823207,95251310,93876799-dd40-4f50-ba4e-0edfb6ea2ed6,138207827,муниципальный округ Академический,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1


In [218]:
mun_dataset[mun_dataset["OBJECTID"] == 31075780]

,OBJECTID,ID,PARENTOBJID,CHANGEID,OKTMO,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH
1620131,31075780,55910706,1406744.0,47547182,45397000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251310.1406744.31075780


А все ли дома есть в этой иерарахии? Нет.

In [220]:
houses_dataset["OBJECTID"].isin(mun_dataset["OBJECTID"]).sum() / len(houses_dataset)

np.float64(0.9979782176680193)

Посмотрим на не попавшие. В административной-то они были все. 

Во-первых, многие из них просроченные.

In [224]:
houses_dataset[~houses_dataset["OBJECTID"].isin(mun_dataset["OBJECTID"])]["ENDDATE"].value_counts().head()

ENDDATE
2019-09-25    201
2019-09-27    119
2019-06-10     88
2079-06-06     81
2019-12-25     32
Name: count, dtype: int64

Во-вторых, все они не активные.

In [233]:
houses_dataset[~houses_dataset["OBJECTID"].isin(mun_dataset["OBJECTID"])]["ISACTIVE"].value_counts()

ISACTIVE
0    703
Name: count, dtype: int64

### Несколько общих проверок

Поищем районы

In [236]:
address_parts_dataset[address_parts_dataset["NAME"].apply(lambda x: "зюзино" in x.lower())]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
9888,1823248,95251351,a4ea3c59-9eec-4931-a785-fb458d93ed46,138207868,муниципальный округ Зюзино,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1


In [237]:
address_parts_dataset[address_parts_dataset["NAME"].apply(lambda x: "раменки" in x.lower())]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
1187,1823154,95251257,fa227bf3-8a4d-4ef5-80ce-7414fd01b4e7,138207774,муниципальный округ Раменки,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
3106,1726638,1402920,0a7df3d5-00d7-4c57-aec6-50cee6295efa,3840615,Раменки,ул,8,1,0,0.0,2016-06-16,1900-01-01,2079-06-06,1,1


Поищем странные улицы

In [238]:
address_parts_dataset[address_parts_dataset["NAME"].apply(lambda x: "рощи" in x.lower())]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
368,1862744,1407043,0457a0af-63d7-45f0-8fe2-1d248db95b3b,156164442,2-й Марьиной Рощи,проезд,8,20,1731686,NaN,2020-08-31,2020-08-31,2079-06-06,1,1
1138,1862738,1408473,e9e758bc-3cbd-4987-8466-d3e15310092c,156164367,10-й Марьиной Рощи,проезд,8,20,1733446,NaN,2020-08-31,2020-08-31,2079-06-06,1,1
1146,1729266,1405091,50176aad-698f-428b-8b44-863e1009c624,3846086,Дубовой Рощи,проезд,8,1,0,0.0,2016-05-24,1900-01-01,2079-06-06,1,1
1169,1862748,1400185,27030e76-05b0-470d-8bd0-39af5b01bcd3,156164447,4-я Марьиной Рощи,ул,8,20,1723328,NaN,2020-08-31,2020-08-31,2079-06-06,1,1
1811,1862747,1403708,3f418142-9996-4e92-b84d-df63ebadc5c3,156164446,4-й Марьиной Рощи,проезд,8,20,1727572,NaN,2020-08-31,2020-08-31,2079-06-06,1,1
2258,1732433,1407631,f17fb4b1-da62-4429-a457-b66b10dfde74,3852239,Марьиной Рощи 15-й,проезд,8,30,1732420,0.0,2019-05-16,2019-05-15,2079-06-06,1,0
2394,1732804,1407941,247c3930-76d0-4b72-9edf-1e752436842e,3852926,Дубовой Рощи,ул,8,1,0,0.0,2016-05-24,1900-01-01,2079-06-06,1,1
2936,1864701,1412892,f6128e27-8bde-4a04-a379-350370ee8efe,156331586,4-й Рощинский,проезд,8,23,1738981,NaN,2020-09-08,2020-09-08,2079-06-06,1,1
3208,1732935,1408048,c07f0f7f-a93f-47e7-90ca-dd6533174b19,3853162,Марьиной Рощи 3-й,проезд,8,1,0,0.0,2016-05-24,1900-01-01,2079-06-06,1,1
3763,1727380,1403489,b9a7c624-24e6-423d-befa-e66e4c2faa53,3842120,Берёзовой Рощи,проезд,8,20,1727311,0.0,2016-06-07,2016-06-07,2079-06-06,1,1


Посмотрим, что с Черёмушками. 

Ох, это бинго из проблем.

Во-первых, е-ё. Во-вторых, кварталы. В-третьих, непонятная улица.

In [243]:
address_parts_dataset[address_parts_dataset["NAME"].apply(lambda x: "черёмушки" in x.lower())]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
12648,1730592,1406121,0549b212-4010-46eb-9bb0-2d0a4f953a32,3848737,Новочерёмушкинская,ул,8,20,1730550,0.0,2016-06-10,2016-06-09,2079-06-06,1,1
19132,1073061,868638,2abea417-e475-439d-b186-18d10ea60efd,2401720,Черёмушки,ул,8,20,1072986,0.0,2016-10-05,2016-06-14,2079-06-06,1,1
20478,1733906,1408786,bb230068-82f9-4306-9a11-6c271360cbce,3855002,Черёмушкинский,проезд,8,20,1733819,0.0,2019-12-15,2016-06-09,2079-06-06,1,1
21630,1864732,1399879,1a329f12-8a04-4e7d-9135-1885355f66e0,156333122,Большая Черёмушкинская,ул,8,23,1722985,NaN,2020-09-08,2020-09-08,2079-06-06,1,1


In [244]:
address_parts_dataset[address_parts_dataset["NAME"].apply(lambda x: "черемушки" in x.lower())]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
954,52131903,1403697,89830603-21f6-43c4-832c-e518b9405e0c,604654869,32а Новые Черемушки,кв-л,7,30,1727651,0.0,2024-10-15,2024-10-15,2079-06-06,1,0
1782,1905013,1408494,4d466794-05e3-419e-8ccf-550fb69a3201,184560370,Новые Черемушки 22-23,кв-л,8,30,1733470,0.0,2021-04-05,2017-11-16,2079-06-06,1,0
2460,1905008,1407246,1d60272e-5108-4838-834e-c59fad72e0a5,184560365,Новые Черемушки 29-30,кв-л,8,30,1731940,0.0,2021-04-05,2017-11-16,2079-06-06,1,0
2464,1905108,1414319,e435b197-2ecf-4fc8-b631-ae0d34655ef2,184560471,Новые Черемушки 24-25,кв-л,8,30,1740874,0.0,2021-04-05,2017-11-16,2079-06-06,1,0
7186,52126882,1411098,5d3d8cc9-d3d2-427d-b2e4-2ab327703749,601215520,ДПК Черемушки,тер.,7,30,1736669,0.0,2024-09-13,2024-09-13,2079-06-06,1,0
9631,1059107,860441,b37451ee-c6fb-4f9d-bb45-781fd83d5b07,2371235,Черемушкинская,ул,8,1,0,0.0,2017-09-07,1900-01-01,2079-06-06,1,1
13041,52100364,874727,d7a8107b-2f18-4bfd-b230-fed13480bc4e,590990357,ДПК Черемушки,тер,7,30,1082863,0.0,2024-02-27,2024-02-27,2079-06-06,1,0
14517,52110173,865343,faa1e403-770e-427c-a061-81e0136f4e14,595714449,СПК Ветеран-Черемушки,тер,8,30,1067420,0.0,2024-05-15,2024-05-15,2079-06-06,1,0
14643,1823214,95251317,b8643c8b-a039-4e7c-a800-db69493f95b8,138207834,муниципальный округ Черемушки,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
22054,52100231,1404480,1e0c9150-df45-4214-b1c4-7c1e7adf49b8,590970545,Новые Черемушки 24а,кв-л,8,30,1728497,0.0,2024-02-26,2024-02-26,2079-06-06,1,0


Улица Черёмушки -- в Щербинке.

In [257]:
object_id = 868638

In [248]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == object_id]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
19132,1073061,868638,2abea417-e475-439d-b186-18d10ea60efd,2401720,Черёмушки,ул,8,20,1072986,0.0,2016-10-05,2016-06-14,2079-06-06,1,1


In [249]:
mun_dataset[mun_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,OKTMO,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH
1834,868638,242830763,865977.0,619680948,45932000,29949231,0,2025-01-27,2025-01-27,2079-06-06,1,1405113.95251374.865977.868638


In [258]:
adm_dataset[adm_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,REGIONCODE,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH,AREACODE,CITYCODE,PLACECODE,PLANCODE,STREETCODE
1960,868638,241226974,865977,619680948,77,97481267,0,2025-01-27,2025-01-27,2079-06-06,1,1405113.169509311.865977.868638,27.0,0.0,7.0,0.0,2.0


In [250]:
address_parts_dataset[address_parts_dataset["OBJECTID"].isin([int(i) for i in "1405113.95251374.865977.868638".split(".")])]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
2558,1068536,865977,1e43ba77-1904-4b1f-a3f6-f23df5c0c5b4,2391799,Ерино,д,6,1,0,0.0,2016-09-26,1900-01-01,2079-06-06,1,1
9068,1729288,1405113,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,3846120,Москва,г,1,1,0,0.0,2018-09-01,1900-01-01,2079-06-06,1,1
14644,52144898,95251374,89184dde-a93f-40ae-b6d5-8645833bddb3,613527978,муниципальный округ Щербинка,вн.тер.г.,3,20,1823271,NaN,2024-12-26,2024-12-26,2079-06-06,1,1
19132,1073061,868638,2abea417-e475-439d-b186-18d10ea60efd,2401720,Черёмушки,ул,8,20,1072986,0.0,2016-10-05,2016-06-14,2079-06-06,1,1


In [260]:
address_parts_dataset[address_parts_dataset["OBJECTID"].isin([int(i) for i in "1405113.169509311.865977.868638".split(".")])]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
2558,1068536,865977,1e43ba77-1904-4b1f-a3f6-f23df5c0c5b4,2391799,Ерино,д,6,1,0,0.0,2016-09-26,1900-01-01,2079-06-06,1,1
9068,1729288,1405113,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,3846120,Москва,г,1,1,0,0.0,2018-09-01,1900-01-01,2079-06-06,1,1
14483,52147930,169509311,2c2f107f-ebf2-46f9-8975-7c2ebf70bd9b,617838291,Щербинка,р-н,2,10,0,0.0,2025-01-16,2025-01-16,2079-06-06,1,1
19132,1073061,868638,2abea417-e475-439d-b186-18d10ea60efd,2401720,Черёмушки,ул,8,20,1072986,0.0,2016-10-05,2016-06-14,2079-06-06,1,1


С кварталом ещё любопытнее. Впрочем, он не активный, можно его попробовать проигнорить.

In [261]:
object_id = 1408494

In [252]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == object_id]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
1782,1905013,1408494,4d466794-05e3-419e-8ccf-550fb69a3201,184560370,Новые Черемушки 22-23,кв-л,8,30,1733470,0.0,2021-04-05,2017-11-16,2079-06-06,1,0


In [253]:
mun_dataset[mun_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,OKTMO,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH
5450,1408494,56601063,95251317.0,44132858,45908000,0,0,1900-01-01,1900-01-01,2079-06-06,0,1405113.95251317.1408494


In [255]:
adm_dataset[adm_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,REGIONCODE,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH,AREACODE,CITYCODE,PLACECODE,PLANCODE,STREETCODE
5611,1408494,96588516,1405113,3854177,77,0,0,1900-01-01,1900-01-01,2079-06-06,0,1405113.1408494,0.0,0.0,0.0,0.0,4127.0


In [254]:
address_parts_dataset[address_parts_dataset["OBJECTID"].isin([int(i) for i in "1405113.95251317.1408494".split(".")])]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
1782,1905013,1408494,4d466794-05e3-419e-8ccf-550fb69a3201,184560370,Новые Черемушки 22-23,кв-л,8,30,1733470,0.0,2021-04-05,2017-11-16,2079-06-06,1,0
9068,1729288,1405113,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,3846120,Москва,г,1,1,0,0.0,2018-09-01,1900-01-01,2079-06-06,1,1
14643,1823214,95251317,b8643c8b-a039-4e7c-a800-db69493f95b8,138207834,муниципальный округ Черемушки,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1


А кто в квартале?

In [265]:
mun_dataset[mun_dataset["PARENTOBJID"] == object_id][["OBJECTID"]].merge(houses_dataset, on="OBJECTID")

,OBJECTID,ID,OBJECTGUID,CHANGEID,HOUSENUM,ADDNUM1,HOUSETYPE,ADDTYPE1,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE,ADDNUM2,ADDTYPE2
0,28715400,70892291,2f76a26b-84d3-4875-bff6-16e8f059e60c,182544464,к1В,1,2.0,2.0,30,17034077,0.0,2021-03-16,1900-01-01,2079-06-06,1,0,None,NaN
1,28715853,70892292,1722c860-b5b3-4154-9332-388fd28adbbc,182544465,None,1,NaN,1.0,30,17034415,0.0,2021-03-16,1900-01-01,2079-06-06,1,0,None,NaN
2,28716803,70553191,e8040565-cb44-4268-b8df-5bc06631b69a,182205357,None,1Г,NaN,1.0,30,17034984,0.0,2021-03-16,1900-01-01,2079-06-06,1,0,None,NaN
3,28717170,70892293,392de842-4c6e-4893-a8e1-8d5c8cbd0ad1,182544466,к5,1,2.0,2.0,30,17035154,0.0,2021-03-16,1900-01-01,2079-06-06,1,0,None,NaN
4,28717927,70553195,ef24b4e2-5f2e-4c13-a61e-9bd4fe2ca632,182205361,None,1В,NaN,1.0,30,17035712,0.0,2021-03-16,1900-01-01,2079-06-06,1,0,None,NaN
5,28718734,70553196,a69452cb-02ba-4962-9a5e-e01be13b12cf,182205362,None,1А,NaN,1.0,30,17036388,0.0,2021-03-16,1900-01-01,2079-06-06,1,0,None,NaN
6,28719152,70553199,3b9dd01c-5c7f-426c-b95a-f72f016f67b2,182205365,None,1Б,NaN,1.0,30,17036725,0.0,2021-03-16,1900-01-01,2079-06-06,1,0,None,NaN


Наконец, было интересно, к каким районам отнесут улицы, проходящие через несколько?

Ленинский проспект уж точно лежит в нескольких.

In [268]:
address_parts_dataset[address_parts_dataset["NAME"].apply(lambda x: "ленинский" in x.lower())]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
9238,1726229,1402585,5f2a1243-a57b-418e-baee-ff76f4993b45,3839686,Ленинский,пр-кт,8,1,0,0.0,2016-05-17,1900-01-01,2079-06-06,1,1


In [269]:
object_id = 1402585

Административно он в Москве, логично.

In [270]:
adm_dataset[adm_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,REGIONCODE,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH,AREACODE,CITYCODE,PLACECODE,PLANCODE,STREETCODE
3396,1402585,114977976,1405113,154399119,77,92425072,0,2020-08-11,2020-08-11,2079-06-06,1,1405113.1402585,0.0,0.0,0.0,0.0,1689.0


In [271]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == 1405113]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
9068,1729288,1405113,0c5b2444-70a0-4932-980c-b4dc0d3f02b5,3846120,Москва,г,1,1,0,0.0,2018-09-01,1900-01-01,2079-06-06,1,1


А муниципально? Только один район?

In [272]:
mun_dataset[mun_dataset["OBJECTID"] == object_id]

,OBJECTID,ID,PARENTOBJID,CHANGEID,OKTMO,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE,PATH
3248,1402585,54625348,95251336.0,125126761,45907000,0,0,1900-01-01,1900-01-01,2079-06-06,1,1405113.95251336.1402585


In [273]:
address_parts_dataset[address_parts_dataset["OBJECTID"] == 95251336]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
2772,1823233,95251336,3e970108-c4e6-4e34-bf81-66cd89d70ff1,138207853,муниципальный округ Теплый Стан,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1


А вот если взять дома по нему и пути к ним -- то найдутся все районы.

In [282]:
address_parts_dataset[address_parts_dataset["OBJECTID"].isin(mun_dataset[mun_dataset["PARENTOBJID"] == object_id]["PATH"].apply(lambda x: int(x.split(".")[1])).unique())]

,ID,OBJECTID,OBJECTGUID,CHANGEID,NAME,TYPENAME,LEVEL,OPERTYPEID,PREVID,NEXTID,UPDATEDATE,STARTDATE,ENDDATE,ISACTUAL,ISACTIVE
1503,1823242,95251345,bee3d09c-1a66-4562-bc87-03412adb6a05,138207862,муниципальный округ Проспект Вернадского,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
2130,1823213,95251316,7c2651cc-5604-4da1-a1a3-6b984e82b8e3,138207833,муниципальный округ Тропарево-Никулино,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
2772,1823233,95251336,3e970108-c4e6-4e34-bf81-66cd89d70ff1,138207853,муниципальный округ Теплый Стан,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
3565,1823135,95251238,48f3ec88-9e75-4f97-acde-cb8b8a752df0,138207755,муниципальный округ Якиманка,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
9200,1823236,95251339,c43b8810-62da-4ff3-a78a-667669718830,138207856,муниципальный округ Гагаринский,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
15197,1823227,95251330,ac38fe1c-643f-4314-826f-1ef5c0e37971,138207847,муниципальный округ Обручевский,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
20191,1823166,95251269,7658016e-ce69-4165-ac67-75bf9df5e020,138207786,муниципальный округ Донской,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1
22204,1823212,95251315,770c4ffa-0fb7-4fd1-b0ac-b688cfdbe166,138207832,муниципальный округ Ломоносовский,вн.тер.г.,3,0,0,0.0,2011-01-01,2011-01-01,2079-06-06,1,1


# Подготовка итоговых данных

Итак, для дальнейшего поиска мы сохраним таблицу с записями вида:
* object_id,
* номер дома,
* тип дома,
* добавочный номер 1,
* добавочный тип 1,
* добавочный номер 2,
* добавочный тип 2,
* название дороги,
* тип дороги,
* is_active.

И другую таблицу:
* objectid,
* тэг.

Добавим пару слов про тэги. Нам хочется, не ограничивая себя, хранить информацию о районе -- и шире, о регионе. При этом мы не хотим сопоставлять каждому дому только один тэг. Более того, хочется иметь возможность добавить в дальнейшем отдельную выгрузку с новым тэгом. Таким образом в отдельной таблице мы будем сопоставлять дома тэгам и применять затем фильтры по тэгам. Использовать мы будем всегда пересечение двух таблиц.

Поскольку нет уверенности, что в разных регионах нам понадобится одинаковая структура, мы не будем реализовывать универсальный скрипт для обработки всех регионов, а ad-hoc соберём данные по интересующему нас в первую очередь региону -- Москве.

## Распаковка нужных данных

In [19]:
house_types_ds = load_dataset(z, root_files[3])
subhouse_types_ds = load_dataset(z, root_files[0])

In [20]:
region_id = "77"
region_files = [f for f in z_files if os.path.dirname(f) == region_id]

In [21]:
%%time
address_parts_dataset = load_dataset(z, region_files[0])
houses_dataset = load_dataset(z, region_files[9])
mun_dataset = load_dataset(z, region_files[11], "OBJECTID")

CPU times: total: 4min 10s
Wall time: 4min 12s


## Соберём данные вместе

Всю информацию о домах:

In [22]:
houses_dataset.shape

(347713, 18)

In [23]:
houses_joint_info_ds = houses_dataset[[
        "OBJECTID", "HOUSENUM", "HOUSETYPE", "ADDNUM1", "ADDTYPE1", "ADDNUM2", "ADDTYPE2", "ISACTIVE"
    ]].merge(
        mun_dataset[["OBJECTID", "PARENTOBJID", "ISACTIVE", "PATH"]].rename(columns={"ISACTIVE": "ISACTIVE_MUN"}),
        on="OBJECTID",
    ).merge(
        house_types_ds[["ID", "SHORTNAME", "ISACTIVE"]].rename(columns={"ISACTIVE": "ISACTIVE_HTYPES", "ID": "HOUSETYPE", "SHORTNAME": "HSHORTNAME"}),
        on="HOUSETYPE",
    ).merge(
        subhouse_types_ds[["ID", "SHORTNAME", "ISACTIVE"]].rename(columns={"ISACTIVE": "ISACTIVE_ADDTYPES1", "ID": "ADDTYPE1", "SHORTNAME": "A1SHORTNAME"}),
        on="ADDTYPE1",
        how="left",
    ).merge(
        subhouse_types_ds[["ID", "SHORTNAME", "ISACTIVE"]].rename(columns={"ISACTIVE": "ISACTIVE_ADDTYPES2", "ID": "ADDTYPE2", "SHORTNAME": "A2SHORTNAME"}),
        on="ADDTYPE2",
        how="left",
    )

In [24]:
len(houses_joint_info_ds) / len(houses_dataset)

0.9961002320879576

Выделим родителей домов и назовём их условно улицами.

In [25]:
streets_info_ds = houses_joint_info_ds[[
        "PARENTOBJID"
    ]].drop_duplicates().merge(
        address_parts_dataset[[
            "OBJECTID", "NAME", "TYPENAME", "LEVEL", "ISACTIVE"
        ]].rename(columns={"ISACTIVE": "ISACTIVE_STREET", "OBJECTID": "PARENTOBJID", "NAME": "STREETNAME", "TYPENAME": "STREETTYPENAME"}),
    )

In [26]:
streets_info_ds.groupby(
    ["LEVEL", "STREETTYPENAME"], as_index=False
)[
    "PARENTOBJID"
].count().sort_values(
    by="PARENTOBJID", ascending=False
).head(10)

,LEVEL,STREETTYPENAME,PARENTOBJID
50,8,ул,3268
11,7,кв-л,1691
38,8,пер,714
44,8,проезд,440
3,6,д,222
19,7,тер.,198
18,7,тер,169
29,8,км,152
42,8,пр-д,79
40,8,пл,75


In [27]:
streets_info_ds[
    streets_info_ds["ISACTIVE_STREET"]==1
].groupby(
    ["LEVEL", "STREETTYPENAME"], as_index=False
)[
    "PARENTOBJID"
].count().sort_values(
    by="PARENTOBJID", ascending=False
).head(10)

,LEVEL,STREETTYPENAME,PARENTOBJID
42,8,ул,3150
9,7,кв-л,1185
30,8,пер,702
36,8,проезд,424
3,6,д,219
24,8,км,144
34,8,пр-д,79
32,8,пл,74
21,8,б-р,73
5,6,п,65


Осталось для домов определить районы:

In [28]:
house_to_all_mun_ds = houses_joint_info_ds.assign(
    ANCESTORID=houses_joint_info_ds.PATH.apply(lambda x: [int(a) for a in x.split(".")][:-1])
)[[
    "OBJECTID", "ANCESTORID"
]].explode(
    "ANCESTORID"
).merge(
    address_parts_dataset[
        address_parts_dataset["LEVEL"] == 3
    ][["OBJECTID", "NAME"]].rename(
        columns={"OBJECTID": "ANCESTORID", "NAME": "AREANAME"}
    ),
    on="ANCESTORID",
)[[
    "OBJECTID", "AREANAME"
]]

## Сохраним нужные таблицы

In [29]:
data_path = os.path.join(os.path.dirname(os.getcwd()), "data")
sep = "\t"

### Таблица с адресами

In [30]:
house_addresses_path = os.path.join(data_path, "77_ha.tsv")

In [31]:
prepared_dataset = houses_joint_info_ds.merge(
    streets_info_ds, on="PARENTOBJID"
)

In [32]:
prepared_dataset["is_active"] = (
    (prepared_dataset[["ISACTIVE", "ISACTIVE_MUN", "ISACTIVE_STREET"]].min(axis=1) == 1)
    & ~(prepared_dataset[['ISACTIVE_HTYPES', 'ISACTIVE_ADDTYPES1', 'ISACTIVE_ADDTYPES2']] == False).max(axis=1)
)

In [33]:
prepared_dataset[[
    "OBJECTID", "STREETNAME", "STREETTYPENAME", "HOUSENUM", "HSHORTNAME", "ADDNUM1", "A1SHORTNAME", "ADDNUM2", "A2SHORTNAME", "is_active"
]].rename(
    columns={
        "OBJECTID": "oid",
        "STREETNAME": "street_name",
        "STREETTYPENAME": "street_type",
        "HOUSENUM": "house_num",
        "HSHORTNAME": "house_type",
        "ADDNUM1": "house_add_1_num",
        "A1SHORTNAME": "house_add_1_type",
        "ADDNUM2": "house_add_2_num",
        "A2SHORTNAME": "house_add_2_type",
    }
).to_csv(house_addresses_path, sep=sep, index=False)

### Таблица с тэгами

In [34]:
house_tags_path = os.path.join(data_path, "77_ht.tsv")

Добавим некоторые вариации.

In [35]:
areas_ds = house_to_all_mun_ds[house_to_all_mun_ds["AREANAME"].apply(lambda x: x.startswith("муниципальный округ"))].copy()

In [36]:
areas_ds["AREANAME"] = areas_ds["AREANAME"].apply(lambda x: x.replace("муниципальный округ", "").strip())

In [37]:
moscow_ds = house_to_all_mun_ds.copy()
moscow_ds["AREANAME"] = "Москва"

In [38]:
pd.concat([
    house_to_all_mun_ds,
    areas_ds,
    moscow_ds,
], ignore_index=True
).rename(
    columns={"OBJECTID": "oid", "AREANAME": "tag"}
).to_csv(house_tags_path, sep=sep, index=False)